In [18]:
!python -m ipykernel install --user --name=myenv

Installed kernelspec myenv in C:\Users\arman\AppData\Roaming\jupyter\kernels\myenv


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import pandas_ta as ta
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from torch.utils.data import Dataset, DataLoader
import warnings

warnings.filterwarnings('ignore')

# تنظیمات جدید
SEED = 42
SEQ_LENGTH = 60
BATCH_SIZE = 64
EPOCHS = 200
torch.manual_seed(SEED)
np.random.seed(SEED)

class ImprovedDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return (
            torch.FloatTensor(self.sequences[idx]),
            torch.FloatTensor([self.targets[idx]]).squeeze()
        )

class StableLSTM(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=64,
            num_layers=1,
            batch_first=True,
            dropout=0.2
        )
        self.fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 1)
        )
        self.init_weights()

    def init_weights(self):
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.xavier_normal_(param)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :]).squeeze()

# بارگذاری و پیش‌پردازش داده
def load_data(file_path):
    df = pd.read_csv(file_path, sep='\t', header=0)
    df['datetime'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])
    df = df.drop(['<DATE>', '<TIME>'], axis=1)
    df.columns = [col.strip('<>').lower() for col in df.columns]
    df = df.set_index('datetime').sort_index()
    
    # ویژگی‌های اضافه
    df['log_return'] = np.log1p(df['close'].pct_change())
    df['volatility'] = df['log_return'].rolling(20).std().fillna(0)
    df.ta.ema(length=20, close='close', append=True)
    df.ta.rsi(length=14, close='close', append=True)
    df = df.fillna(0).replace([np.inf, -np.inf], 0)
    return df

# آموزش با تنظیمات جدید
def train(model, train_loader, test_loader):
    optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-6)
    scheduler = optim.lr_scheduler.CyclicLR(
        optimizer,
        base_lr=1e-6,
        max_lr=1e-4,
        step_size_up=50,
        cycle_momentum=False
    )
    criterion = nn.MSELoss()
    
    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_loss += loss.item()
        
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                test_loss += criterion(outputs, labels).item()
        
        scheduler.step()
        avg_train_loss = train_loss/len(train_loader)
        avg_test_loss = test_loss/len(test_loader)
        print(f'Epoch {epoch+1}/{EPOCHS} | Train Loss: {avg_train_loss:.6f} | Test Loss: {avg_test_loss:.6f}')

# اجرای اصلی
if __name__ == "__main__":
    df = load_data('XAUUSD-I_M30.txt')
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df.values)
    
    # ایجاد دنباله‌ها
    sequences, targets = [], []
    for i in range(len(scaled_data)-SEQ_LENGTH):
        sequences.append(scaled_data[i:i+SEQ_LENGTH])
        targets.append(scaled_data[i+SEQ_LENGTH, df.columns.get_loc('close')])
    
    # تقسیم داده
    tscv = TimeSeriesSplit(n_splits=5)
    for train_idx, test_idx in tscv.split(sequences):
        train_seq, test_seq = np.array(sequences)[train_idx], np.array(sequences)[test_idx]
        train_tgt, test_tgt = np.array(targets)[train_idx], np.array(targets)[test_idx]
    
    train_loader = DataLoader(ImprovedDataset(train_seq, train_tgt), batch_size=BATCH_SIZE)
    test_loader = DataLoader(ImprovedDataset(test_seq, test_tgt), batch_size=BATCH_SIZE)
    
    model = StableLSTM(input_size=df.shape[1])
    train(model, train_loader, test_loader)

Epoch 1/200
Train Loss: 6333209.981579 | Test Loss: 9896099.921053
RMSE: 3145.572219 | R²: -391.8825 | Accuracy (±1%): 0.00%
LR: 1.00e-04
----------------------------------------------------------------------
Epoch 2/200
Train Loss: 6331374.125000 | Test Loss: 9890179.605263
RMSE: 3144.632917 | R²: -391.6479 | Accuracy (±1%): 0.00%
LR: 1.00e-04
----------------------------------------------------------------------
Epoch 3/200
Train Loss: 6330309.140789 | Test Loss: 9884527.500000
RMSE: 3143.735343 | R²: -391.4238 | Accuracy (±1%): 0.00%
LR: 1.00e-04
----------------------------------------------------------------------
Epoch 4/200
Train Loss: 6329487.668421 | Test Loss: 9878674.789474
RMSE: 3142.805300 | R²: -391.1917 | Accuracy (±1%): 0.00%
LR: 1.00e-04
----------------------------------------------------------------------
Epoch 5/200
Train Loss: 6328712.686842 | Test Loss: 9873183.184211
RMSE: 3141.932712 | R²: -390.9739 | Accuracy (±1%): 0.00%
LR: 1.00e-04
--------------------------